In [8]:
import numpy as np
import pandas as pd
from scipy.stats import sem

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

import pickle

from util import T, mean_err, M_pi, M_K, Z_V, N_K, C1, C2
# from main_diagrams import typeI_typeII_t_seps, calc_amp_multiple_trajs, calc_amp_sum_tv

# trajs = range(2100, 2150, 10)
trajs = sorted([int(f[:4]) for f in os.listdir('./Kaon_results_new_strat/')])

# trajs = [t for t in trajs if t<1290]  # 1010 - 1270 amplitude is 20% larger than 1900 - 2150

print('trajs:', trajs)
print(f'Number of trajectories: {len(trajs)}')

trajs: [1010, 1030, 1040, 1050, 1070, 1080, 1090, 1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1270, 1280, 1290, 1300, 1310, 1320, 1330, 1350, 1360, 1370, 1380, 1390, 1400, 1410, 1420, 1430, 1440, 1450, 1460, 1470, 1480, 1490, 1500, 1510, 1520, 1530, 1540, 1550, 1560, 1570, 1580, 1590, 1600, 1610, 1630, 1640, 1650, 1660, 1670, 1680, 1690, 1700, 1710, 1720, 1730, 1740, 1750, 1760, 1770, 1780, 1790, 1800, 1810, 1820, 1840, 1850, 1860, 1870, 1890, 1900, 1910, 1930, 1940, 1950, 1960, 1970, 1990, 2000, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2100, 2110, 2120, 2130, 2140, 2150, 2160, 2170, 2180, 2190, 2200, 2210, 2220, 2230, 2240, 2250]
Number of trajectories: 115


In [9]:
coefficients = {
    'typeV_D1Q1': 5/9, 'typeV_D1Q2': -5/9,  # disconnected # Error is too large (about 100% - 200%) with 35 configurations # add this later
    'typeV_D2Q1': 1/9, 'typeV_D2Q2': -1/9,
    
    'sBar_d_T3D1': -5/9, 'sBar_d_T3D2': -1/9,   # disconnected
}

diagrams = list(coefficients.keys())

Q1_diagrams = [d for d in diagrams if d.endswith('Q1')] 
Q2_diagrams = [d for d in diagrams if d.endswith('Q2')] 
sBar_d_diagrams = [d for d in diagrams if d.startswith('sBar_d')]

print(f'Q1_diagrams: {Q1_diagrams}')
print(f'Q2_diagrams: {Q2_diagrams}')
print(f'sBar_d_diagrams: {sBar_d_diagrams}')

data = {}
all_trajs = sorted([int(f[:4]) for f in os.listdir('./Kaon_results_new_strat/')])
for traj in all_trajs:
    data[traj] = pickle.load(open(f'./Kaon_results_new_strat/{traj}.pkl', 'rb'))

Q1_diagrams: ['typeV_D1Q1', 'typeV_D2Q1']
Q2_diagrams: ['typeV_D1Q2', 'typeV_D2Q2']
sBar_d_diagrams: ['sBar_d_T3D1', 'sBar_d_T3D2']


In [10]:
d = diagrams[0]

hadron_coef = Z_V**2 * 2 * M_K / N_K
amp_Q1 = np.stack([sum([data[traj][d] * coefficients[d] * hadron_coef for d in Q1_diagrams])
                   for traj in all_trajs]).real # amp_Q1[traj][R][tv]
amp_Q2 = np.stack([sum([data[traj][d] * coefficients[d] * hadron_coef for d in Q2_diagrams])
                   for traj in all_trajs]).real # amp_Q2[traj][R][tv]

amp_Hw = 1. / np.sqrt(2) * (C1 * amp_Q1 + C2 * amp_Q2) # 1. / np.sqrt(2) is the factor in K_L

In [11]:
max_R = 10
amp_sum_R = amp_Hw[:, :max_R].sum(axis=1) # amp_sum_R[traj][tv]
amp_mean = amp_sum_R.mean(axis=0) # amp_mean[tv]
amp_err = sem(amp_sum_R, axis=0)

# tv_min, tv_max = -4, 8
tv_min, tv_max = -4, 6
y = np.concatenate([amp_mean[tv_min:], amp_mean[:tv_max+1]])
err_y = np.concatenate([amp_err[tv_min:], amp_err[:tv_max+1]])

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(tv_min, tv_max+1)), y=y, 
                         error_y=dict(type='data', array=err_y))
)

fig.add_shape(dict(type='line', line_dash='dot',
      y0= -0.02047, y1=-0.02047,
      x0= tv_min, x1=tv_max))

fig.update_layout(title_text='', width=600, height=400)
fig.update_layout(title='Amplitude before subtracting pion contribution')
fig.update_xaxes(title_text='t_v')
fig.update_yaxes(title_text='Amplitude')
# fig.write_image('amp_vs_tv_before_pion.pdf')
fig.show()

In [12]:
max_R = 10
total_amp_mean, total_amp_err = {}, {}

for max_R in range(21):
    amp_sum_R = amp_Hw[:, :max_R].sum(axis=1) # amp_sum_R[traj][tv]
#     total_amp = amp_sum_R[:, 0:3].sum(axis=1) # total_amp[traj]  # [0, 2]
    total_amp = amp_sum_R[:, 1:5].sum(axis=1) # total_amp[traj]  # [0, 2]
#     total_amp = amp_sum_R[:, 1:2].sum(axis=1) # total_amp[traj]    # [0, 1]
    
    total_amp_mean[max_R], total_amp_err[max_R] = total_amp.mean(), sem(total_amp)
    
total_amp_mean = pd.Series(total_amp_mean)
total_amp_err = pd.Series(total_amp_err)

In [13]:
amp_Hw.shape

(115, 21, 64)

In [14]:
fig = px.scatter(total_amp_mean, error_y=total_amp_err)
fig.update_yaxes(title='Amplitude')
fig.update_xaxes(title='Max R')
fig.write_image('typeV_amp_vs_R.pdf')
fig

[17600:1205/105937.510302:ERROR:buffer_manager.cc(488)] [.DisplayCompositor]GL ERROR :GL_INVALID_OPERATION : glBufferData: <- error from previous GL command


In [15]:
total_amp_mean[14], total_amp_err[14]

(0.06837467220078482, 0.01812402610740399)